In [1]:
# dependencies
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import create_engine, func
from sqlalchemy import create_engine, inspect

In [2]:
phytoplankton = pd.read_csv("data/phytoplankton.csv")
zooplankton =  pd.read_csv("data/zooplankton.csv")
phytoplankton_color = pd.read_csv("data/phytoplankton_ci.csv")

In [3]:

phytoplankton.head()

,FID,id,Route,Latitude,Longitude,SampleDateUTC,Year,Month,Day,Time_24hr,...,caab_code,taxon_start_date,taxon_per_m3,phyto_comments,acknowledgements,first_occurrence,parent_taxon_name,training,comments,geom
0,soop_auscpr_phyto_flat_map.fid--404dafa0_17a9a...,30880720,ABAB,-37.9138,135.3310,2020-02-27T02:01:35Z,2020,2,27,02:01:35,...,50566000.0,NaN,3333.3,NaN,Wallenius Wilhelmsen,NaN,NaN,NaN,NaN,POINT (135.33099 -37.91378)
1,soop_auscpr_phyto_flat_map.fid--404dafa0_17a9a...,30880722,ABAB,-37.9138,135.3310,2020-02-27T02:01:35Z,2020,2,27,02:01:35,...,52030054.0,NaN,666.7,*Noted as present in the zoop count*,Wallenius Wilhelmsen,NaN,NaN,NaN,NaN,POINT (135.33099 -37.91378)
2,soop_auscpr_phyto_flat_map.fid--404dafa0_17a9a...,30880721,ABAB,-37.9138,135.3310,2020-02-27T02:01:35Z,2020,2,27,02:01:35,...,52030900.0,2011-05-17,666.7,*Noted as present in the zoop count*,Wallenius Wilhelmsen,2011-09-20,Tripos spp.,"Internal training Pru Bonhma, Gustaaf Hallegraeff",NaN,POINT (135.33099 -37.91378)
3,soop_auscpr_phyto_flat_map.fid--404dafa0_17a9a...,30880726,ABAB,-37.8766,134.9125,2020-02-27T03:10:21Z,2020,2,27,03:10:21,...,99530018.0,2011-05-17,6666.7,NaN,Wallenius Wilhelmsen,2011-09-15,Pseudo-nitzschia spp.,"Internal training Pru Bonhma, Gustaaf Hallegraeff",NaN,POINT (134.91247 -37.87656)
4,soop_auscpr_phyto_flat_map.fid--404dafa0_17a9a...,30880727,ABAB,-37.8766,134.9125,2020-02-27T03:10:21Z,2020,2,27,03:10:21,...,NaN,NaN,3333.3,NaN,Wallenius Wilhelmsen,NaN,NaN,NaN,NaN,POINT (134.91247 -37.87656)


In [4]:
#rounding coordinates
phytoplankton = phytoplankton.round(2)
phytoplankton.head()

,FID,id,Route,Latitude,Longitude,SampleDateUTC,Year,Month,Day,Time_24hr,...,caab_code,taxon_start_date,taxon_per_m3,phyto_comments,acknowledgements,first_occurrence,parent_taxon_name,training,comments,geom
0,soop_auscpr_phyto_flat_map.fid--404dafa0_17a9a...,30880720,ABAB,-37.91,135.33,2020-02-27T02:01:35Z,2020,2,27,02:01:35,...,50566000.0,NaN,3333.3,NaN,Wallenius Wilhelmsen,NaN,NaN,NaN,NaN,POINT (135.33099 -37.91378)
1,soop_auscpr_phyto_flat_map.fid--404dafa0_17a9a...,30880722,ABAB,-37.91,135.33,2020-02-27T02:01:35Z,2020,2,27,02:01:35,...,52030054.0,NaN,666.7,*Noted as present in the zoop count*,Wallenius Wilhelmsen,NaN,NaN,NaN,NaN,POINT (135.33099 -37.91378)
2,soop_auscpr_phyto_flat_map.fid--404dafa0_17a9a...,30880721,ABAB,-37.91,135.33,2020-02-27T02:01:35Z,2020,2,27,02:01:35,...,52030900.0,2011-05-17,666.7,*Noted as present in the zoop count*,Wallenius Wilhelmsen,2011-09-20,Tripos spp.,"Internal training Pru Bonhma, Gustaaf Hallegraeff",NaN,POINT (135.33099 -37.91378)
3,soop_auscpr_phyto_flat_map.fid--404dafa0_17a9a...,30880726,ABAB,-37.88,134.91,2020-02-27T03:10:21Z,2020,2,27,03:10:21,...,99530018.0,2011-05-17,6666.7,NaN,Wallenius Wilhelmsen,2011-09-15,Pseudo-nitzschia spp.,"Internal training Pru Bonhma, Gustaaf Hallegraeff",NaN,POINT (134.91247 -37.87656)
4,soop_auscpr_phyto_flat_map.fid--404dafa0_17a9a...,30880727,ABAB,-37.88,134.91,2020-02-27T03:10:21Z,2020,2,27,03:10:21,...,NaN,NaN,3333.3,NaN,Wallenius Wilhelmsen,NaN,NaN,NaN,NaN,POINT (134.91247 -37.87656)


In [5]:
#putting lat and long into one column

#phytoplankton["coordinates"] = "[" + phytoplankton["Latitude"].astype(str) +", " + phytoplankton["Longitude"].astype(str) + "]"

In [6]:
# get list of unique routes
column_values = phytoplankton[["Route"]].values.ravel()
unique_routes =  pd.unique(column_values)

print(unique_routes)

['ABAB' 'ABFR' 'ADAB' 'ADME' 'ANHO' 'AUHO' 'BRER' 'BRFI' 'BRGL' 'BRNC'
 'BRPK' 'BRSY' 'BUNE' 'CAGL' 'EDMI' 'FRAN' 'FRBO' 'FRSY' 'GLTO' 'HOAN'
 'HOFR' 'HOSO' 'HOSY' 'IIOE' 'MEAB' 'MEAD' 'MEFR' 'MEPK' 'NCME' 'NEBU'
 'PKME' 'SOHO' 'SYBR' 'SYHO' 'SYME' 'TINT' 'TOCA']


In [7]:
#total count of unique values

item_counts = phytoplankton["Route"].value_counts()
print(item_counts)

HOAN    15806
ANHO     6284
BRSY     3470
BRPK     2647
SYME     2098
PKME     1952
MEAD     1777
CAGL     1084
ADAB      969
ABFR      743
HOFR      735
MEAB      630
FRBO      554
GLTO      432
HOSY      406
FRAN      390
BRNC      364
BRGL      335
IIOE      315
BRER      299
SYBR      199
BUNE      191
MEFR      190
NCME      182
FRSY      172
SYHO      160
AUHO      147
TINT      144
HOSO      114
EDMI      105
NEBU       98
ABAB       90
MEPK       83
SOHO       55
BRFI       55
TOCA       49
ADME       33
Name: Route, dtype: int64


In [8]:
#for loop to check which routes to drop

for route in unique_routes:
    print(route)
    route_filter = phytoplankton.loc[phytoplankton['Route'] == route]
    print(route_filter["Year"].value_counts())
    print("***************************************************")

ABAB
2020    90
Name: Year, dtype: int64
***************************************************
ABFR
2014    300
2016    220
2017    136
2015     87
Name: Year, dtype: int64
***************************************************
ADAB
2014    488
2016    330
2017     92
2015     59
Name: Year, dtype: int64
***************************************************
ADME
2019    33
Name: Year, dtype: int64
***************************************************
ANHO
2010    1432
2009    1327
2008    1148
2018    1132
2011     702
2013     336
2017     103
2016      79
2019      24
2007       1
Name: Year, dtype: int64
***************************************************
AUHO
2011    147
Name: Year, dtype: int64
***************************************************
BRER
2014    299
Name: Year, dtype: int64
***************************************************
BRFI
2012    55
Name: Year, dtype: int64
***************************************************
BRGL
2014    281
2015     54
Name: Year, dtype: int64
*******

In [9]:
#drop unneeded columns for phytoplankton
phytoplankton = phytoplankton.drop(["id",
                                    "Time_24hr",
                                    "FID",
                                    "SampleDateUTC",
#                                    "Latitude",
#                                    "Longitude",
                                    "Month",
                                    "Day",
                                    "family",
                                    "genus",
                                    "taxon_group",
                                    "species",
                                    "caab_code",
                                   "taxon_start_date",
                                    "taxon_name",
                                   "phyto_comments",
                                   "acknowledgements",
                                   "first_occurrence",
                                   "parent_taxon_name",
                                   "training",
                                   "comments",
                                    "geom"
                                  ],
                                  axis=1)

In [10]:
phytoplankton.head()

,Route,Latitude,Longitude,Year,taxon_per_m3
0,ABAB,-37.91,135.33,2020,3333.3
1,ABAB,-37.91,135.33,2020,666.7
2,ABAB,-37.91,135.33,2020,666.7
3,ABAB,-37.88,134.91,2020,6666.7
4,ABAB,-37.88,134.91,2020,3333.3


In [11]:
# #have only four main routes for sample size due to data limits

# phyto_main_routes = phytoplankton.loc[((phytoplankton['Route'] == "ANHO") |
#         (phytoplankton["Route"] == 'BRSY')|
#         (phytoplankton["Route"] == 'HOAN')|
#         (phytoplankton["Route"] == 'PKME')|
#         (phytoplankton["Route"] == 'MEAD')), :]

In [12]:
phyto_main_routes = phytoplankton.drop(["Route"], axis=1)
phyto_main_routes = phyto_main_routes.reset_index().drop(["index"],axis=1)
phyto_main_routes.head()

,Latitude,Longitude,Year,taxon_per_m3
0,-37.91,135.33,2020,3333.3
1,-37.91,135.33,2020,666.7
2,-37.91,135.33,2020,666.7
3,-37.88,134.91,2020,6666.7
4,-37.88,134.91,2020,3333.3


In [13]:
#phyto_main_routes = phyto_main_routes.drop(["index"],axis=1)
phyto_main_routes.head()

,Latitude,Longitude,Year,taxon_per_m3
0,-37.91,135.33,2020,3333.3
1,-37.91,135.33,2020,666.7
2,-37.91,135.33,2020,666.7
3,-37.88,134.91,2020,6666.7
4,-37.88,134.91,2020,3333.3


In [14]:
#create empty geojson object
# read csv forEach row in csv append geometry

#Geojson.append


In [15]:
#ANHO, BRSY, HOAN, MEAD, PKME

In [16]:
phytoplankton.count

<bound method DataFrame.count of       Route  Latitude  Longitude  Year  taxon_per_m3
0      ABAB    -37.91     135.33  2020        3333.3
1      ABAB    -37.91     135.33  2020         666.7
2      ABAB    -37.91     135.33  2020         666.7
3      ABAB    -37.88     134.91  2020        6666.7
4      ABAB    -37.88     134.91  2020        3333.3
...     ...       ...        ...   ...           ...
43352  TOCA    -16.80     145.92  2013         666.7
43353  TOCA    -16.80     145.92  2013         666.7
43354  TOCA    -16.80     145.92  2013         666.7
43355  TOCA    -16.80     145.92  2013         666.7
43356  TOCA    -16.80     145.92  2013         666.7

[43357 rows x 5 columns]>

In [17]:
phyto_main_routes.count


<bound method DataFrame.count of        Latitude  Longitude  Year  taxon_per_m3
0        -37.91     135.33  2020        3333.3
1        -37.91     135.33  2020         666.7
2        -37.91     135.33  2020         666.7
3        -37.88     134.91  2020        6666.7
4        -37.88     134.91  2020        3333.3
...         ...        ...   ...           ...
43352    -16.80     145.92  2013         666.7
43353    -16.80     145.92  2013         666.7
43354    -16.80     145.92  2013         666.7
43355    -16.80     145.92  2013         666.7
43356    -16.80     145.92  2013         666.7

[43357 rows x 4 columns]>

# Zooplankton

In [18]:
#rounding coordinates
zooplankton = zooplankton.round(2)
zooplankton.head()

,FID,id,Route,Latitude,Longitude,SampleDateUTC,Year,Month,Day,Time_24hr,...,aphiaid,taxon_start_date,taxon_per_m3,zoop_comments,acknowledgements,first_occurrence,parent_taxon_name,training,comments,geom
0,soop_auscpr_zoop_flat_map.fid--620ea8f0_17a9a6...,60974299,ABAB,-37.91,135.33,2020-02-27T02:01:35Z,2020,2,27,02:01:35,...,1100.0,2008-09-29,2.00,NaN,Wallenius Wilhelmsen,NaN,NaN,NaN,NaN,POINT (135.33099 -37.91378)
1,soop_auscpr_zoop_flat_map.fid--620ea8f0_17a9a6...,60974379,ABAB,-37.91,135.33,2020-02-27T02:01:35Z,2020,2,27,02:01:35,...,2081.0,2008-09-29,0.67,NaN,Wallenius Wilhelmsen,NaN,NaN,NaN,NaN,POINT (135.33099 -37.91378)
2,soop_auscpr_zoop_flat_map.fid--620ea8f0_17a9a6...,60974380,ABAB,-37.91,135.33,2020-02-27T02:01:35Z,2020,2,27,02:01:35,...,1410.0,2013-03-22,1.33,NaN,Wallenius Wilhelmsen,NaN,NaN,NaN,NaN,POINT (135.33099 -37.91378)
3,soop_auscpr_zoop_flat_map.fid--620ea8f0_17a9a6...,60974378,ABAB,-37.91,135.33,2020-02-27T02:01:35Z,2020,2,27,02:01:35,...,106485.0,2008-09-29,4.00,NaN,Wallenius Wilhelmsen,NaN,NaN,NaN,NaN,POINT (135.33099 -37.91378)
4,soop_auscpr_zoop_flat_map.fid--620ea8f0_17a9a6...,60974296,ABAB,-37.88,134.91,2020-02-27T03:10:21Z,2020,2,27,03:10:21,...,106652.0,2011-07-08,0.67,NaN,Wallenius Wilhelmsen,2010-11-18,Oithona Grp3 (pointy head) f,External training Dave McKinnon,Identified when plumes or setae on gential seg...,POINT (134.91247 -37.87656)


In [19]:
#coordinates column
#zooplankton["coordinates"] = "[" + zooplankton["Latitude"].astype(str) +", " + zooplankton["Longitude"].astype(str) + "]"

In [20]:
for col in zooplankton.columns:
    print(col)

FID
id
Route
Latitude
Longitude
SampleDateUTC
Year
Month
Day
Time_24hr
taxon_name
family
genus
species
sex
life_stage
taxon_group
taxon_eco_group
caab_code
aphiaid
taxon_start_date
taxon_per_m3
zoop_comments
acknowledgements
first_occurrence
parent_taxon_name
training
comments
geom


In [21]:
#drop unneeded columns for zooplankton
zooplankton = zooplankton.drop(["id",
                                "FID",
                                "Time_24hr",
                                "SampleDateUTC",
#                                "Latitude",
#                                "Longitude",
                                "family",
                                "Month",
                                "Day",
                                "genus",
                                "taxon_group",
                                "species",
                                "sex",
                                  "life_stage",
                                    "caab_code",
                                    "aphiaid",  
                                   "taxon_start_date",
                                   "zoop_comments",
                                   "acknowledgements",
                                   "first_occurrence",
                                "taxon_name",
                                   "parent_taxon_name",
                                   "training",
                                   "comments",
                                    "taxon_eco_group",
                                "geom"
                                  ],
                                  axis=1)

In [22]:
zooplankton.head()

,Route,Latitude,Longitude,Year,taxon_per_m3
0,ABAB,-37.91,135.33,2020,2.00
1,ABAB,-37.91,135.33,2020,0.67
2,ABAB,-37.91,135.33,2020,1.33
3,ABAB,-37.91,135.33,2020,4.00
4,ABAB,-37.88,134.91,2020,0.67


In [23]:
zooplankton.count

<bound method DataFrame.count of        Route  Latitude  Longitude  Year  taxon_per_m3
0       ABAB    -37.91     135.33  2020          2.00
1       ABAB    -37.91     135.33  2020          0.67
2       ABAB    -37.91     135.33  2020          1.33
3       ABAB    -37.91     135.33  2020          4.00
4       ABAB    -37.88     134.91  2020          0.67
...      ...       ...        ...   ...           ...
111409  TOCA    -16.80     145.92  2013          0.67
111410  TOCA    -16.80     145.92  2013          1.33
111411  TOCA    -16.80     145.92  2013          0.67
111412  TOCA    -16.80     145.92  2013          0.67
111413  TOCA    -16.80     145.92  2013          0.67

[111414 rows x 5 columns]>

In [24]:
column_values = zooplankton[["Route"]].values.ravel()
unique_routes =  pd.unique(column_values)

print(unique_routes)

['ABAB' 'ABFR' 'ADAB' 'ADME' 'ANHO' 'AUHO' 'BRER' 'BRFI' 'BRGL' 'BRNC'
 'BRPK' 'BRSY' 'BUNE' 'CAGL' 'FRBO' 'FRSY' 'GLTO' 'HOAN' 'HOFR' 'HOSO'
 'HOSY' 'IIOE' 'MEAB' 'MEAD' 'MEFR' 'MEPK' 'NCME' 'NEBU' 'PKME' 'SOHO'
 'SYBR' 'SYHO' 'SYME' 'TINT' 'TOCA']


In [25]:
#for loop to check which routes to drop

for route in unique_routes:
    print(route)
    route_filter = zooplankton.loc[zooplankton['Route'] == route]
    print(route_filter["Year"].value_counts())
    print("***************************************************")

ABAB
2020    297
Name: Year, dtype: int64
***************************************************
ABFR
2016    800
2015    661
2017    537
2014    433
Name: Year, dtype: int64
***************************************************
ADAB
2014    1390
2016     593
2017     303
2015     216
Name: Year, dtype: int64
***************************************************
ADME
2019    355
Name: Year, dtype: int64
***************************************************
ANHO
2007    2343
2010    2193
2013    2025
2009    1211
2008     918
2011     867
2018     293
2016     263
2017     192
2019      64
Name: Year, dtype: int64
***************************************************
AUHO
2011    982
Name: Year, dtype: int64
***************************************************
BRER
2014    766
Name: Year, dtype: int64
***************************************************
BRFI
2012    234
Name: Year, dtype: int64
***************************************************
BRGL
2014    1027
2015     265
Name: Year, dtype: int6

In [26]:
#ANHO, BRSY, HOAN, MEAD, PKME

In [27]:
#zoo_main_routes = zooplankton.loc[((zooplankton['Route'] == "ANHO") |
#         (zooplankton["Route"] == 'BRSY')|
#         (zooplankton["Route"] == 'HOAN')|
#         (zooplankton["Route"] == 'PKME')|
#         (zooplankton["Route"] == 'MEAD')), :]

In [28]:
zoo_main_routes.head()

NameError: name 'zoo_main_routes' is not defined

In [29]:
#drop route column
zoo_main_routes = zooplankton.drop(["Route"], axis =1)
zoo_main_routes = zoo_main_routes.reset_index().drop(["index"],axis=1)
zoo_main_routes.head()

,Latitude,Longitude,Year,taxon_per_m3
0,-37.91,135.33,2020,2.00
1,-37.91,135.33,2020,0.67
2,-37.91,135.33,2020,1.33
3,-37.91,135.33,2020,4.00
4,-37.88,134.91,2020,0.67


In [30]:
# save to csv, remember to delete index in csv to save space
zoo_main_routes.to_csv("clean_data/zooplankton_clean.csv")
phyto_main_routes.to_csv("clean_data/phytoplankton_clean.csv")

In [31]:
zoo_main_routes.count

<bound method DataFrame.count of         Latitude  Longitude  Year  taxon_per_m3
0         -37.91     135.33  2020          2.00
1         -37.91     135.33  2020          0.67
2         -37.91     135.33  2020          1.33
3         -37.91     135.33  2020          4.00
4         -37.88     134.91  2020          0.67
...          ...        ...   ...           ...
111409    -16.80     145.92  2013          0.67
111410    -16.80     145.92  2013          1.33
111411    -16.80     145.92  2013          0.67
111412    -16.80     145.92  2013          0.67
111413    -16.80     145.92  2013          0.67

[111414 rows x 4 columns]>

In [32]:
for col in phytoplankton_color.columns:
    print(col)

FID
measurement
route_code
start_port
end_port
route_frequency
route_start_date
vessel_name
trip_code
colour_index
colour
pci_comments
acknowledgements
TIME
geom
latitude
longitude


In [33]:
#drop unneeded columns for phytoplankton color index
phytoplankton_color = phytoplankton_color.drop(["FID",
                                                "start_port",
                                                "end_port",
                                                "trip_code",
                                                "colour_index",
                                                "measurement",
                                               "acknowledgements",
                                               "route_frequency",
                                               "route_start_date",
                                               "pci_comments",
                                                "geom",
                                                  ],
                                                  axis=1)

In [34]:
phytoplankton_color

,route_code,vessel_name,colour,TIME,latitude,longitude
0,ABAB,Salome IMO,VERY_PALE_GREEN,2020-02-27T02:01:35Z,-37.9138,135.3310
1,ABAB,Salome IMO,VERY_PALE_GREEN,2020-02-27T02:18:47Z,-37.9046,135.2263
2,ABAB,Salome IMO,VERY_PALE_GREEN,2020-02-27T02:35:58Z,-37.8954,135.1217
3,ABAB,Salome IMO,VERY_PALE_GREEN,2020-02-27T02:53:10Z,-37.8860,135.0171
4,ABAB,Salome IMO,VERY_PALE_GREEN,2020-02-27T03:10:21Z,-37.8766,134.9125
...,...,...,...,...,...,...
27627,TOCA,Kweichow IMO 9070694,VERY_PALE_GREEN,2016-10-17T18:52:05Z,-17.2799,146.1455
27628,TOCA,Kweichow IMO 9070694,VERY_PALE_GREEN,2016-10-17T19:18:59Z,-17.1994,146.1217
27629,TOCA,Kweichow IMO 9070694,VERY_PALE_GREEN,2016-10-17T19:45:53Z,-17.1189,146.0979
27630,TOCA,Kweichow IMO 9070694,VERY_PALE_GREEN,2016-10-17T20:12:47Z,-17.0384,146.0742


In [35]:
# Create an engine for the plankton.sqlite database
engine = create_engine("sqlite:///planktontestmain4.sqlite", echo=False)
conn = engine.connect()

In [36]:
# add csv to data base function
# def read_sql(data):
#     data.to_sql(f"'{data}'", conn, if_exists = "append")


In [37]:
# # add csv to data base
# read_sql(phytoplankton)
# read_sql(zooplankton)
# read_sql(phytoplankton_color)

In [38]:
phyto_main_routes.to_sql("phytoplankton", conn,)
zoo_main_routes.to_sql("zooplankton", conn,)
phytoplankton_color.to_sql("phytoplankton_color_index", conn,)

In [39]:
# Reflect Database into ORM classes
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

[]

In [40]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [57]:
# Reflect Database into ORM classes
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

[]

In [58]:
# Use `engine.execute` to select and display the first 10 rows from the station table
engine.execute('SELECT * FROM zooplankton LIMIT 5').fetchall()

[(0, -37.91, 135.33, 2020, 'COPEPOD', 2.0),
 (1, -37.91, 135.33, 2020, 'NON-COPEPOD', 0.67),
 (2, -37.91, 135.33, 2020, 'NON-COPEPOD', 1.33),
 (3, -37.91, 135.33, 2020, 'COPEPOD', 4.0),
 (4, -37.88, 134.91, 2020, 'COPEPOD', 0.67)]

In [59]:
# Query phytoplankton Records in the the Database
data = pd.read_sql("SELECT * FROM phytoplankton", conn)

In [60]:
data.head()

,index,Latitude,Longitude,Year,taxon_group,taxon_per_m3
0,0,-37.91,135.33,2020,Ciliate,3333.3
1,1,-37.91,135.33,2020,Dinoflagellate,666.7
2,2,-37.91,135.33,2020,Dinoflagellate,666.7
3,3,-37.88,134.91,2020,Pennate diatom,6666.7
4,4,-37.88,134.91,2020,Dinoflagellate,3333.3
